# Decorators

## Overview - Day 1

A decorator can add behavior to a function:
- Functions are passed into a decorator.
- The decorator can do something to the function before or after the function runs.
- Returns the newly decorated object.

---

### Decorator Example

#### Import the required modules

In [1]:
# `wraps` is not required, although it is preferred
from functools import wraps
import time

#### Create a basic template for defining a decorator

In [5]:
def my_first_decorator(function):
    @wraps(function)
    # *args and **kwargs are placeholders for arguments/keyword arguments from the function passed to the decorator
    def wrapper(*args, **kwargs):
        # Do something before running the function passed to the decorator
        # Call the function passed to the decorator
        result = function(*args, **kwargs)
        # Do something after running the function passed to the decorator
        return result
    # return `wrapper` which is the decorated function
    return wrapper

#### Use the decorator to wrap the function

The decorator syntax uses the @ symbol to prefix the decorator function.

In [3]:
@my_first_decorator
def my_function(args):
    pass

The decorator syntax is is syntactic sugar for:

In [8]:
# This is NOT the common way to use a decorator
def my_new_function(args):
    pass

my_new_function = my_first_decorator(my_new_function)

---

### `*args` and `**kwargs` parameters in decorators

Python function arguments include:
- **positional**
    - Mandatory and have no default values.
- **keyword**
    - Optional and have defaults.
- **arbitrary**
    - `*args` becomes a `tuple` of all remaining (not explicitly defined) **positional** arguments.
    - `*kwargs` becomes a `dict` of all remaining (not explicitly defined) **keyword** arguments.

>Note: The parameter names `*args` and `**kwargs` are arbitrary.
> - These could instead be `*seeds` and `**nuts`.

In [13]:
# Example
def test_args(first_name, *args, **kwargs):
    ''' The parameters *args and **kwargs are optional and capture any
        unspecified positional and keyword arguments, respectively.
    '''

    # Return a list of all arguments
    return_value = {
        'first_name': first_name,
        'args': args,
        'kwargs': kwargs
    }

    return return_value

''' When calling the function, any keyword arguments whether defined explicitly or not, must come after any positional arguments
'''
test_args('Tim', 'W', 'Hull', age=41, state='Oregon')

{'first_name': 'Tim',
 'args': ('W', 'Hull'),
 'kwargs': {'age': 41, 'state': 'Oregon'}}

#### Create a decorator to display arguments

- Display `*args` before calling the function passed to the decorator.
- Display `**kwargs` after calling the function passed to the decorator.

In [32]:
from functools import wraps

def display_args(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        print('\nThis decorator displays *args values before calling the function:'
              f'{args}\n')

        # Call the function passed to the decorator
        result = function(*args, **kwargs)

        print('And also displays **kwargs values after calling the function:'
              f'{kwargs}\n')

        return result

    # Return wrapper as a decorated function
    return wrapper

In [33]:
@display_args
def send_args(*args, **kwargs):
    print('\tThis is a decorated function\n')

send_args('Tim', 'Hull', hair='blonde', eyes='blue')


This decorator displays *args values before calling the function:('Tim', 'Hull')

	This is a decorated function

And also displays **kwargs values after calling the function:{'hair': 'blonde', 'eyes': 'blue'}



---

## Review - Day 2

### Create a new decorator and apply it to a new function

In [1]:
# Import modules
from functools import wraps
from time import ctime, time

In [2]:
# Create decorator function
def add_times(wrapped_function):
    @wraps(wrapped_function)

    # Define a wrapper function
    def wrapper(*args, **kwargs):
        # Display the epoch before the wrapped function
        print(f'\nThe current epoch time is {time()}')

        # Call the wrapped function
        result = wrapped_function(*args, **kwargs)

        # Display the time after the wrapped function
        print(f'The current time is {ctime()}\n')

        # return the result of the wrapepd function
        return result

    # Return the result of the wrapper function
    return wrapper

In [3]:
# Define a new function using the decorator
@add_times
def my_name(name):
    print(f'\tMy name is {name}')

In [4]:
# Call the wrapped function
my_name('Tim')


The current epoch time is 1628967894.996227
	My name is Tim
The current time is Sat Aug 14 19:04:54 2021



---

### Write a Time-It Decorator

In [9]:
# Import Modules
from functools import wraps
from time import time
from typing import Union

In [16]:
# Create the decorator function
def time_it(function):
    ''' This function is a decorator which displays the total time it takes
        for the decorated function to run.

        Args:
            function (function):
                Decorated function.

        Returns:
            None.
    '''

    ''' Call the @wraps decorator on the decorated function.
        This preserves the docstring in the wrapped function.
    '''
    @wraps(function)
    def wrapper(*args, **kwargs):

        # Capture the time before the wrapped function starts
        start_time = time()

        # Call the decorated function
        result = function(*args, **kwargs)

        # Capture the time after the wrapped function ends
        end_time = time()
        total_time = round(start_time - end_time, 5)

        # Display the time the wrapped function tool to run
        print(f'The "{function.__name__}" function took {total_time} to run.')

        # return the result of the wrapped function
        return result

    # Return the result of the wrapper function
    return wrapper


In [17]:
# Create a decorated function to time
@time_it
def get_exponent(num: Union[int, float], exp: int = 1) -> Union[int, float]:
    ''' Determine the exponent of a number.

        Args:
            num (int or float):
                Base number.
            exp (int):
                Exponent to apply to `num`.

        Returns:
            exponent (int or float):
                Result of the exponent operation.
    '''

    # Determine the exponent
    exponent = num ** exp

    # Display the result
    print(f'The exponent of {num} to the {exp} power is {exponent}.')

    return exponent

In [24]:
# Call the decorated function
exp = get_exponent(10, 10)

The exponent of 10 to the 10 power is 10000000000.
The "get_exponent" function took -0.00017 to run.


### Test the use of the @wraps decorator

`@wraps` should ensure the docstring of the wrapped function is preserved

In [25]:
# Look up the docstring for the wrapped function
help(get_exponent)

Help on function get_exponent in module __main__:

get_exponent(num: Union[int, float], exp: int = 1) -> Union[int, float]
    Determine the exponent of a number.
    
    Args:
        num (int or float):
            Base number.
        exp (int):
            Exponent to apply to `num`.
    
    Returns:
        exponent (int or float):
            Result of the exponent operation.



### Redfine the decorator function without `@wraps`

In [26]:
# Create the decorator function
def time_it(function):
    ''' This function is a decorator which displays the total time it takes
        for the decorated function to run.

        Args:
            function (function):
                Decorated function.

        Returns:
            None.
    '''

    ''' Call the @wraps decorator on the decorated function.
        This preserves the docstring in the wrapped function.
    '''
    def wrapper(*args, **kwargs):

        # Capture the time before the wrapped function starts
        start_time = time()

        # Call the decorated function
        result = function(*args, **kwargs)

        # Capture the time after the wrapped function ends
        end_time = time()
        total_time = round(start_time - end_time, 5)

        # Display the time the wrapped function tool to run
        print(f'The "{function.__name__}" function took {total_time} to run.')

        # return the result of the wrapped function
        return result

    # Return the result of the wrapper function
    return wrapper


### Redefine the wrapped function

Notice that the absense of the `@wraps` decorator in the `time_it` (decorator) function prevents the dosctring from being available in the wrapped function (`get_exponent`)

In [28]:
# Create a decorated function to time
@time_it
def get_exponent(num: Union[int, float], exp: int = 1) -> Union[int, float]:
    ''' Determine the exponent of a number.

        Args:
            num (int or float):
                Base number.
            exp (int):
                Exponent to apply to `num`.

        Returns:
            exponent (int or float):
                Result of the exponent operation.
    '''

    # Determine the exponent
    exponent = num ** exp

    # Display the result
    print(f'The exponent of {num} to the {exp} power is {exponent}.')

    return exponent

help(get_exponent)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)

